<a href="https://colab.research.google.com/github/Pedro-HMC-Martins/Inovalab_project/blob/main/inova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface

In [ ]:
!pip install transformers

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install unidecode

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install torch

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import shutil
import sys
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, default_data_collator, BertModel

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
from unidecode import unidecode

In [ ]:
df_comentario = pd.read_csv('/content/Comentarios_Dock.csv', sep=';')
df_comentario.head()

,Comentário,Sentimento
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo
1,Muitas mudanças em muito pouco tempo,negativo
2,"Trabalho remoto para sempre, empresa em cresci...",positivo
3,Salário baixo e existe pessoas entrando com sa...,negativo
4,"Bons beneficios, salário razoavel com mercado,...",positivo


In [ ]:
df_comentario.shape

(5320, 2)

In [ ]:
df_estrelas = pd.read_csv('/content/Estrelas_comentarios.csv' ,  sep = ';')
df_estrelas.head()

,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,5.0,5.0,4.0,3.0,4.0,3.0
1,1.0,1.0,NaN,NaN,1.0,1.0
2,3.0,3.0,NaN,3.0,3.0,3.0
3,1.0,1.0,1.0,2.0,2.0,1.0
4,4.0,4.0,4.0,3.0,4.0,4.0


In [ ]:
df_estrelas.shape

(2670, 6)

In [ ]:
def concat_dfs(df1,df2):
  df2 = pd.DataFrame(pd.np.repeat(df2.values,2,axis = 0), columns=df2.columns)
  df_final = pd.concat([df1, df2], axis = 1)
  return df_final

In [ ]:
df_comentario = df_comentario.iloc[:-80, :]
df_estrelas = df_estrelas.iloc[:-50, :]
print(df_comentario.shape)
print(df_estrelas.shape)

(5240, 2)
(2620, 6)


In [ ]:
df_final = concat_dfs(df_comentario, df_estrelas)

<ipython-input-28-bd2066c36925>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df2 = pd.DataFrame(pd.np.repeat(df2.values,2,axis = 0), columns=df2.columns)


In [ ]:
df_final.shape

(5240, 8)

In [ ]:
df_final.head()

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudanças em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
2,"Trabalho remoto para sempre, empresa em cresci...",positivo,1.0,1.0,NaN,NaN,1.0,1.0
3,Salário baixo e existe pessoas entrando com sa...,negativo,1.0,1.0,NaN,NaN,1.0,1.0
4,"Bons beneficios, salário razoavel com mercado,...",positivo,3.0,3.0,NaN,3.0,3.0,3.0


In [ ]:
df_final = df_final.dropna()

In [ ]:
df_final['Comentário'][7]

'A empresa se perdeu, tocadores de bumbo que são mais palestrantes do linkedin do que líderes de times. A empresa se tornou uma das piores para se trabalhar no mercado, tem tocadores de bumbo que eram analistas de atendimento ao cliente e com uma carreira meteórica de um ex "ditador" deixou e criou o verdadeiro circo. Fazem demissões em massa com movimentos sem alarde, só no mês de Setembro, a folha diminuiu cerca de 120 pessoas. Tocador de bumbo de Ithealth é o mais perdido de toda a empresa, só que consegue enganar bem os times, pois é bom de papo e articulador de parcerias, mas técnicamente não é evoluido e não sabe fazer gestão de pessoas, 1:1 com ele é a pior situação que existe, pois são faz piadinhas e cria situações ruins.'

In [ ]:
df_final.shape

(4034, 8)

In [ ]:
df_final

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudanças em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
6,Era uma empresa ótima para se trabalhar.,positivo,1.0,1.0,1.0,2.0,2.0,1.0
7,"A empresa se perdeu, tocadores de bumbo que sã...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,Ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5225,Ainda iniciando planos de carreira,negativo,5.0,5.0,5.0,4.0,5.0,5.0
5236,"Entrosamento da equipe, remuneração, benefícios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,Modelo de gestão de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0
5238,Pessoas incríveis e ambiente ótimo !,positivo,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
def remover_acentos(texto):
  return unidecode(texto)

df_final['Comentário'] = df_final['Comentário'].apply(remover_acentos)

In [ ]:
frase_referencia = "Nao tenho nada contra"

def similaridade(comentario, frase_referencia):
  return fuzz.token_set_ratio(comentario, frase_referencia)

limiar_similaridade = 40
df_final = df_final[df_final['Comentário'].apply(lambda x : similaridade(x,frase_referencia)) < limiar_similaridade]
df_final

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudancas em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
6,Era uma empresa otima para se trabalhar.,positivo,1.0,1.0,1.0,2.0,2.0,1.0
7,"A empresa se perdeu, tocadores de bumbo que sa...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,Ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5224,Total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5236,"Entrosamento da equipe, remuneracao, beneficios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,Modelo de gestao de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0
5238,Pessoas incriveis e ambiente otimo !,positivo,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
df_final = df_final.drop(6)
df_final


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudancas em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
7,"A empresa se perdeu, tocadores de bumbo que sa...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,Ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
9,Empresa nova buscando equilibrio de despesas e...,negativo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5224,Total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5236,"Entrosamento da equipe, remuneracao, beneficios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,Modelo de gestao de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0
5238,Pessoas incriveis e ambiente otimo !,positivo,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
condicao = (df_final['Sentimento'] == 'negativo') & \
           (df_final.iloc[:, 2:8].mean(axis=1) > 4.4) & \
           (~(df_final.iloc[:, 2:8] == 2.0).any(axis=1)) & \
           (~(df_final.iloc[:, 2:8] == 1.0).any(axis=1))

df_final = df_final[~condicao]

df_final

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudancas em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
7,"A empresa se perdeu, tocadores de bumbo que sa...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,Ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
9,Empresa nova buscando equilibrio de despesas e...,negativo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,Muito Boa para trabalhar e crescer profissiona...,positivo,3.0,4.0,4.0,5.0,4.0,4.0
5224,Total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5236,"Entrosamento da equipe, remuneracao, beneficios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,Modelo de gestao de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0


In [ ]:
condicao_positivo = (df_final['Sentimento'] == 'positivo') & \
                    (df_final.iloc[:, 2:8].mean(axis=1) < 2.5) & \
                    (~(df_final.iloc[:, 2:8] == 4.0).any(axis=1)) & \
                    (~(df_final.iloc[:, 2:8] == 5.0).any(axis=1))
df_final = df_final[~condicao_positivo]
df_final

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudancas em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
7,"A empresa se perdeu, tocadores de bumbo que sa...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,Ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
9,Empresa nova buscando equilibrio de despesas e...,negativo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,Muito Boa para trabalhar e crescer profissiona...,positivo,3.0,4.0,4.0,5.0,4.0,4.0
5224,Total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5236,"Entrosamento da equipe, remuneracao, beneficios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,Modelo de gestao de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0


In [ ]:
contagem_sentimento = df_final['Sentimento'].value_counts()

print(contagem_sentimento)

positivo    1694
negativo    1028
Name: Sentimento, dtype: int64


In [ ]:
df_final.to_csv('Dataset_final', index = False)

In [ ]:
df_final['Comentário'] = df_final['Comentário'].str.lower()

<ipython-input-44-9829d998c771>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Comentário'] = df_final['Comentário'].str.lower()


In [ ]:
df_final

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,muitas mudancas em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
7,"a empresa se perdeu, tocadores de bumbo que sa...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
9,empresa nova buscando equilibrio de despesas e...,negativo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,muito boa para trabalhar e crescer profissiona...,positivo,3.0,4.0,4.0,5.0,4.0,4.0
5224,total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5236,"entrosamento da equipe, remuneracao, beneficios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,modelo de gestao de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0


## Positivo : 0 , Negativo : 1

In [ ]:
df_final['Sentimento'] = df_final['Sentimento'].map({'positivo': 0, 'negativo': 1})
df_final

<ipython-input-46-0f5276b9276d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Sentimento'] = df_final['Sentimento'].map({'positivo': 0, 'negativo': 1})


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"cultura, reconhecimento de melhorarias, todos ...",0,5.0,5.0,4.0,3.0,4.0,3.0
1,muitas mudancas em muito pouco tempo,1,5.0,5.0,4.0,3.0,4.0,3.0
7,"a empresa se perdeu, tocadores de bumbo que sa...",1,1.0,1.0,1.0,2.0,2.0,1.0
8,ambiente de trabalho amistoso e 100% em casa.,0,4.0,4.0,4.0,3.0,4.0,4.0
9,empresa nova buscando equilibrio de despesas e...,1,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,muito boa para trabalhar e crescer profissiona...,0,3.0,4.0,4.0,5.0,4.0,4.0
5224,total liberdade para com suas ideias e forma d...,0,5.0,5.0,5.0,4.0,5.0,5.0
5236,"entrosamento da equipe, remuneracao, beneficios.",0,5.0,3.0,5.0,5.0,5.0,3.0
5237,modelo de gestao de pessoas,1,5.0,3.0,5.0,5.0,5.0,3.0


In [ ]:
class sentimento_dataset(Dataset):
  def __init__(self,textos,rotulos,tokenizer,max_len):
    self.textos = textos
    self.rotulos = rotulos
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.textos)

  def __getitem__(self, index):
    texto = str(self.textos[index])
    rotulo = self.rotulos[index]

    encoding = self.tokenizer.encode_plus(
        texto,
        add_special_tokens = True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask = True,
        return_tensors = 'pt',
        truncation = True
    )

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(rotulo, dtype=torch.long)
    }


In [ ]:
from sklearn.model_selection import train_test_split

df_final = df_final.sample(frac=1, random_state=67)
df_final

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
1393,nenhuma situacao que eu me recorde,1,4.0,4.0,5.0,4.0,4.0,4.0
2536,pessoas que trabalham na pp realmente gostam ...,0,5.0,5.0,5.0,4.0,5.0,3.0
1017,nao valorizam os funcionarios: nao existe clar...,1,3.0,2.0,2.0,2.0,4.0,2.0
4317,salarios desiguais e dificuldades em promocoes,1,4.0,5.0,5.0,3.0,3.0,3.0
3175,ninguem se importa com sua carreira ou desenvo...,1,4.0,1.0,1.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...
1406,ambiente e qualidade de vida,0,5.0,5.0,5.0,3.0,5.0,3.0
1486,"otimo ambiente de trabalho, beneficios e oport...",0,5.0,5.0,5.0,5.0,5.0,5.0
3687,pouca estruturacao das informacoes para apurac...,1,5.0,3.0,5.0,5.0,4.0,4.0
3128,"salario acima do mercado, participacao nos lucros",0,4.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
df_sentimento = df_final[['Comentário','Sentimento']]
df_sentimento

,Comentário,Sentimento
1393,nenhuma situacao que eu me recorde,1
2536,pessoas que trabalham na pp realmente gostam ...,0
1017,nao valorizam os funcionarios: nao existe clar...,1
4317,salarios desiguais e dificuldades em promocoes,1
3175,ninguem se importa com sua carreira ou desenvo...,1
...,...,...
1406,ambiente e qualidade de vida,0
1486,"otimo ambiente de trabalho, beneficios e oport...",0
3687,pouca estruturacao das informacoes para apurac...,1
3128,"salario acima do mercado, participacao nos lucros",0


In [ ]:
df_sentimento_train, df_sentimento_temp = train_test_split(df_sentimento, test_size = 0.2, random_state=42)
df_sentimento_eval, df_sentimento_test = train_test_split(df_sentimento_temp, test_size = 0.5, random_state=42)

In [ ]:
df_sentimento_train.shape

(2177, 2)

In [ ]:
df_sentimento_eval.shape

(272, 2)

In [ ]:
df_sentimento_test.shape

(273, 2)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
max_len = 256

train_dataset = sentimento_dataset(textos = df_sentimento_train.Comentário.to_numpy(),
                                   rotulos = df_sentimento_train.Sentimento.to_numpy(),
                                   tokenizer = tokenizer,
                                   max_len = max_len)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [ ]:
eval_dataset = sentimento_dataset(textos=df_sentimento_eval.Comentário.to_numpy(),
                                  rotulos=df_sentimento_eval.Sentimento.to_numpy(),
                                  tokenizer=tokenizer,
                                  max_len=max_len)

In [ ]:
test_dataset = sentimento_dataset(textos=df_sentimento_test.Comentário.to_numpy(),
                                  rotulos=df_sentimento_test.Sentimento.to_numpy(),
                                  tokenizer=tokenizer,
                                  max_len=max_len)

In [ ]:
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)

model.to('cuda')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=25,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
meu_comentario = "uma verdadeira selva, as pessoas falam se comendo, ninguém consegue ser educado lá. Partem pra ignorância, a melhor coisa que eu fiz foi sair daquela espelunca, minha cabeça já tava girando e pedindo socorro"

inputs = tokenizer.encode_plus(
    meu_comentario,
    add_special_tokens=True,
    max_length=max_len,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

input_ids = inputs['input_ids'].to('cuda')
attention_mask = inputs['attention_mask'].to('cuda')

model.eval()

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

logits = outputs.logits
predicted_label = logits.argmax(dim=-1).cpu().numpy()[0]

if predicted_label == 0:
    print("Sentimento Positivo")
else:
    print("Sentimento Negativo")


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Inova_modelo_save")
tokenizer.save_pretrained("/content/drive/MyDrive/Inova_modelo_save")

('/content/drive/MyDrive/Inova_modelo_save/tokenizer_config.json',
 '/content/drive/MyDrive/Inova_modelo_save/special_tokens_map.json',
 '/content/drive/MyDrive/Inova_modelo_save/vocab.txt',
 '/content/drive/MyDrive/Inova_modelo_save/added_tokens.json')

In [ ]:
model.to('cpu')

NameError: ignored

In [ ]:
df_departamentos_novos = pd.read_csv('/content/Comentarios_Dock_novos.csv', sep = ';')
df_departamentos_novos

,Comentário,Sentimento
0,A empresa realmente disponibiliza para o funci...,positivo
1,"Para tanto beneficio, tanta coisa boa, entao n...",negativo
2,"Adoro os valores da companhia, que impactam a ...",positivo
3,"O ambiente de trabalho é dinâmico, o que eu ac...",negativo
4,"Adoro meu trabalho, meus colegas são incríveis...",positivo
...,...,...
6735,Carreira estanque e pouca perspectiva de aumen...,negativo
6736,Ambiente muito bom para se trabalhar,positivo
6737,"desorganização, todos os colaboradores fazem u...",negativo
6738,empresa bem posicionada no mercado de comunicação,positivo


In [ ]:
df_estrelas_novos = pd.read_csv('/content/Estrelas_comentarios_novos.csv', sep = ';')
df_estrelas_novos

,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,4.0,5.0,5.0,3.0,5.0,5.0
1,3.0,5.0,5.0,5.0,4.0,5.0
2,5.0,3.0,4.0,3.0,5.0,5.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,4.0,3.0,5.0,4.0,5.0
...,...,...,...,...,...,...
3365,2.0,4.0,NaN,3.0,5.0,3.0
3366,NaN,NaN,NaN,NaN,NaN,NaN
3367,3.0,4.0,NaN,2.0,5.0,NaN
3368,4.0,1.0,NaN,1.0,3.0,1.0


In [ ]:
df_estrelas_antigo = pd.read_csv('/content/Estrelas_comentarios.csv', sep = ';')
df_estrelas_antigo

,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,5.0,5.0,4.0,3.0,4.0,3.0
1,1.0,1.0,NaN,NaN,1.0,1.0
2,3.0,3.0,NaN,3.0,3.0,3.0
3,1.0,1.0,1.0,2.0,2.0,1.0
4,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...
2665,5.0,5.0,NaN,4.0,3.0,4.0
2666,NaN,NaN,NaN,NaN,NaN,NaN
2667,5.0,3.0,NaN,5.0,5.0,3.0
2668,4.0,4.0,NaN,3.0,5.0,4.0


In [ ]:
df_departamentos_antigos = pd.read_csv('/content/Comentarios_Dock.csv', sep= ';')
df_departamentos_antigos

,Comentário,Sentimento
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo
1,Muitas mudanças em muito pouco tempo,negativo
2,"Trabalho remoto para sempre, empresa em cresci...",positivo
3,Salário baixo e existe pessoas entrando com sa...,negativo
4,"Bons beneficios, salário razoavel com mercado,...",positivo
...,...,...
5315,Não tenho nada contra à empresa.,negativo
5316,Centrada no bem estar do colaborador,positivo
5317,"Não ache nada contra, é excelente",negativo
5318,Muito boa empresa e com bons benefícios.,positivo


In [ ]:
df_departamentos_antigos = df_departamentos_antigos.iloc[:-80, :]
df_estrelas_antigo = df_estrelas_antigo.iloc[:-50, :]

In [ ]:
df_final_parte1 = concat_dfs(df_departamentos_antigos,df_estrelas_antigo)
df_final_parte1

<ipython-input-28-bd2066c36925>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df2 = pd.DataFrame(pd.np.repeat(df2.values,2,axis = 0), columns=df2.columns)


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudanças em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
2,"Trabalho remoto para sempre, empresa em cresci...",positivo,1.0,1.0,NaN,NaN,1.0,1.0
3,Salário baixo e existe pessoas entrando com sa...,negativo,1.0,1.0,NaN,NaN,1.0,1.0
4,"Bons beneficios, salário razoavel com mercado,...",positivo,3.0,3.0,NaN,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...
5235,"Sem pontos negativos, pois ainda estou em perí...",negativo,5.0,5.0,NaN,5.0,5.0,5.0
5236,"Entrosamento da equipe, remuneração, benefícios.",positivo,5.0,3.0,5.0,5.0,5.0,3.0
5237,Modelo de gestão de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0
5238,Pessoas incríveis e ambiente ótimo !,positivo,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
df_final_parte1 = df_final_parte1.dropna()
df_final_parte1.shape

(4034, 8)

In [ ]:
df_final_parte1 = df_final_parte1[df_final_parte1['Comentário'].apply(lambda x : similaridade(x,frase_referencia)) < limiar_similaridade]
df_final_parte1

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"Cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,Muitas mudanças em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
6,Era uma empresa ótima para se trabalhar.,positivo,1.0,1.0,1.0,2.0,2.0,1.0
7,"A empresa se perdeu, tocadores de bumbo que sã...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,Ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,Muito Boa para trabalhar e crescer profissiona...,positivo,3.0,4.0,4.0,5.0,4.0,4.0
5224,Total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5237,Modelo de gestão de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0
5238,Pessoas incríveis e ambiente ótimo !,positivo,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
df_final_parte1['Comentário'] = df_final_parte1['Comentário'].str.lower()
df_final_parte1

<ipython-input-66-36102de33ab9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_parte1['Comentário'] = df_final_parte1['Comentário'].str.lower()


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"cultura, reconhecimento de melhorarias, todos ...",positivo,5.0,5.0,4.0,3.0,4.0,3.0
1,muitas mudanças em muito pouco tempo,negativo,5.0,5.0,4.0,3.0,4.0,3.0
6,era uma empresa ótima para se trabalhar.,positivo,1.0,1.0,1.0,2.0,2.0,1.0
7,"a empresa se perdeu, tocadores de bumbo que sã...",negativo,1.0,1.0,1.0,2.0,2.0,1.0
8,ambiente de trabalho amistoso e 100% em casa.,positivo,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,muito boa para trabalhar e crescer profissiona...,positivo,3.0,4.0,4.0,5.0,4.0,4.0
5224,total liberdade para com suas ideias e forma d...,positivo,5.0,5.0,5.0,4.0,5.0,5.0
5237,modelo de gestão de pessoas,negativo,5.0,3.0,5.0,5.0,5.0,3.0
5238,pessoas incríveis e ambiente ótimo !,positivo,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
df_final_parte1['Sentimento'] = df_final_parte1['Sentimento'].map({'positivo': 0, 'negativo': 1})
df_final_parte1

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"cultura, reconhecimento de melhorarias, todos ...",0,5.0,5.0,4.0,3.0,4.0,3.0
1,muitas mudanças em muito pouco tempo,1,5.0,5.0,4.0,3.0,4.0,3.0
6,era uma empresa ótima para se trabalhar.,0,1.0,1.0,1.0,2.0,2.0,1.0
7,"a empresa se perdeu, tocadores de bumbo que sã...",1,1.0,1.0,1.0,2.0,2.0,1.0
8,ambiente de trabalho amistoso e 100% em casa.,0,4.0,4.0,4.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
5222,muito boa para trabalhar e crescer profissiona...,0,3.0,4.0,4.0,5.0,4.0,4.0
5224,total liberdade para com suas ideias e forma d...,0,5.0,5.0,5.0,4.0,5.0,5.0
5237,modelo de gestão de pessoas,1,5.0,3.0,5.0,5.0,5.0,3.0
5238,pessoas incríveis e ambiente ótimo !,0,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
def transformar_valor(sentimento, valor):
    if sentimento == 0:
        return 1 if valor >= 4.0 else 0
    else:
        return 1 if valor <= 3.0 else 0

In [ ]:
colunas_para_transformar = ['Work/Life Balance', 'Culture & Values', 'Diversity & Inclusion',
                            'Career Opportunities', 'Compensation and Benefits', 'Senior Management']

In [ ]:
for coluna in colunas_para_transformar:
  df_final_parte1[coluna] = df_final_parte1.apply(lambda row: transformar_valor(row['Sentimento'], row[coluna]), axis=1)
df_final_parte1

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"cultura, reconhecimento de melhorarias, todos ...",0,1,1,1,0,1,0
1,muitas mudanças em muito pouco tempo,1,0,0,0,1,0,1
6,era uma empresa ótima para se trabalhar.,0,0,0,0,0,0,0
7,"a empresa se perdeu, tocadores de bumbo que sã...",1,1,1,1,1,1,1
8,ambiente de trabalho amistoso e 100% em casa.,0,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...
5222,muito boa para trabalhar e crescer profissiona...,0,0,1,1,1,1,1
5224,total liberdade para com suas ideias e forma d...,0,1,1,1,1,1,1
5237,modelo de gestão de pessoas,1,0,1,0,0,0,1
5238,pessoas incríveis e ambiente ótimo !,0,1,1,1,1,1,1


In [ ]:
df_final_parte2 = concat_dfs(df_departamentos_novos, df_estrelas_novos)
df_final_parte2

<ipython-input-28-bd2066c36925>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df2 = pd.DataFrame(pd.np.repeat(df2.values,2,axis = 0), columns=df2.columns)


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,A empresa realmente disponibiliza para o funci...,positivo,4.0,5.0,5.0,3.0,5.0,5.0
1,"Para tanto beneficio, tanta coisa boa, entao n...",negativo,4.0,5.0,5.0,3.0,5.0,5.0
2,"Adoro os valores da companhia, que impactam a ...",positivo,3.0,5.0,5.0,5.0,4.0,5.0
3,"O ambiente de trabalho é dinâmico, o que eu ac...",negativo,3.0,5.0,5.0,5.0,4.0,5.0
4,"Adoro meu trabalho, meus colegas são incríveis...",positivo,5.0,3.0,4.0,3.0,5.0,5.0
...,...,...,...,...,...,...,...,...
6735,Carreira estanque e pouca perspectiva de aumen...,negativo,3.0,4.0,NaN,2.0,5.0,NaN
6736,Ambiente muito bom para se trabalhar,positivo,4.0,1.0,NaN,1.0,3.0,1.0
6737,"desorganização, todos os colaboradores fazem u...",negativo,4.0,1.0,NaN,1.0,3.0,1.0
6738,empresa bem posicionada no mercado de comunicação,positivo,3.0,2.0,NaN,3.0,2.0,3.0


In [ ]:
df_final_parte2 = df_final_parte2.dropna()
df_final_parte2

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,A empresa realmente disponibiliza para o funci...,positivo,4.0,5.0,5.0,3.0,5.0,5.0
1,"Para tanto beneficio, tanta coisa boa, entao n...",negativo,4.0,5.0,5.0,3.0,5.0,5.0
2,"Adoro os valores da companhia, que impactam a ...",positivo,3.0,5.0,5.0,5.0,4.0,5.0
3,"O ambiente de trabalho é dinâmico, o que eu ac...",negativo,3.0,5.0,5.0,5.0,4.0,5.0
4,"Adoro meu trabalho, meus colegas são incríveis...",positivo,5.0,3.0,4.0,3.0,5.0,5.0
...,...,...,...,...,...,...,...,...
6679,realmente muito longe a empresa,negativo,5.0,5.0,5.0,5.0,5.0,5.0
6700,"Ações, comida no escritório, colegas interessa...",positivo,4.0,5.0,2.0,4.0,4.0,4.0
6701,Cada vez mais complicado e lento no realizar p...,negativo,4.0,5.0,2.0,4.0,4.0,4.0
6728,"Ambiente descontraídos, vários benefícios, bom...",positivo,5.0,5.0,5.0,3.0,4.0,4.0


In [ ]:
df_final_parte2['Comentário'] = df_final_parte2['Comentário'].str.lower()
df_final_parte2

<ipython-input-73-fd96bd003607>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_parte2['Comentário'] = df_final_parte2['Comentário'].str.lower()


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,a empresa realmente disponibiliza para o funci...,positivo,4.0,5.0,5.0,3.0,5.0,5.0
1,"para tanto beneficio, tanta coisa boa, entao n...",negativo,4.0,5.0,5.0,3.0,5.0,5.0
2,"adoro os valores da companhia, que impactam a ...",positivo,3.0,5.0,5.0,5.0,4.0,5.0
3,"o ambiente de trabalho é dinâmico, o que eu ac...",negativo,3.0,5.0,5.0,5.0,4.0,5.0
4,"adoro meu trabalho, meus colegas são incríveis...",positivo,5.0,3.0,4.0,3.0,5.0,5.0
...,...,...,...,...,...,...,...,...
6679,realmente muito longe a empresa,negativo,5.0,5.0,5.0,5.0,5.0,5.0
6700,"ações, comida no escritório, colegas interessa...",positivo,4.0,5.0,2.0,4.0,4.0,4.0
6701,cada vez mais complicado e lento no realizar p...,negativo,4.0,5.0,2.0,4.0,4.0,4.0
6728,"ambiente descontraídos, vários benefícios, bom...",positivo,5.0,5.0,5.0,3.0,4.0,4.0


In [ ]:
df_final_parte2['Sentimento'] = df_final_parte2['Sentimento'].map({'positivo': 0, 'negativo': 1})
df_final_parte2

<ipython-input-74-525ed8b3d9ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_parte2['Sentimento'] = df_final_parte2['Sentimento'].map({'positivo': 0, 'negativo': 1})


,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,a empresa realmente disponibiliza para o funci...,0,4.0,5.0,5.0,3.0,5.0,5.0
1,"para tanto beneficio, tanta coisa boa, entao n...",1,4.0,5.0,5.0,3.0,5.0,5.0
2,"adoro os valores da companhia, que impactam a ...",0,3.0,5.0,5.0,5.0,4.0,5.0
3,"o ambiente de trabalho é dinâmico, o que eu ac...",1,3.0,5.0,5.0,5.0,4.0,5.0
4,"adoro meu trabalho, meus colegas são incríveis...",0,5.0,3.0,4.0,3.0,5.0,5.0
...,...,...,...,...,...,...,...,...
6679,realmente muito longe a empresa,1,5.0,5.0,5.0,5.0,5.0,5.0
6700,"ações, comida no escritório, colegas interessa...",0,4.0,5.0,2.0,4.0,4.0,4.0
6701,cada vez mais complicado e lento no realizar p...,1,4.0,5.0,2.0,4.0,4.0,4.0
6728,"ambiente descontraídos, vários benefícios, bom...",0,5.0,5.0,5.0,3.0,4.0,4.0


In [ ]:
for coluna in colunas_para_transformar:
  df_final_parte2[coluna] = df_final_parte2.apply(lambda row: transformar_valor(row['Sentimento'], row[coluna]), axis=1)
df_final_parte2

<ipython-input-75-678a568fa70a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_parte2[coluna] = df_final_parte2.apply(lambda row: transformar_valor(row['Sentimento'], row[coluna]), axis=1)
<ipython-input-75-678a568fa70a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_parte2[coluna] = df_final_parte2.apply(lambda row: transformar_valor(row['Sentimento'], row[coluna]), axis=1)
<ipython-input-75-678a568fa70a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,a empresa realmente disponibiliza para o funci...,0,1,1,1,0,1,1
1,"para tanto beneficio, tanta coisa boa, entao n...",1,0,0,0,1,0,0
2,"adoro os valores da companhia, que impactam a ...",0,0,1,1,1,1,1
3,"o ambiente de trabalho é dinâmico, o que eu ac...",1,1,0,0,0,0,0
4,"adoro meu trabalho, meus colegas são incríveis...",0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...
6679,realmente muito longe a empresa,1,0,0,0,0,0,0
6700,"ações, comida no escritório, colegas interessa...",0,1,1,0,1,1,1
6701,cada vez mais complicado e lento no realizar p...,1,0,0,1,0,0,0
6728,"ambiente descontraídos, vários benefícios, bom...",0,1,1,1,0,1,1


In [ ]:
df_final_parte2 = df_final_parte2[df_final_parte2['Comentário'].apply(lambda x : similaridade(x,frase_referencia)) < limiar_similaridade]
df_final_parte2

,Comentário,Sentimento,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,a empresa realmente disponibiliza para o funci...,0,1,1,1,0,1,1
2,"adoro os valores da companhia, que impactam a ...",0,0,1,1,1,1,1
3,"o ambiente de trabalho é dinâmico, o que eu ac...",1,1,0,0,0,0,0
4,"adoro meu trabalho, meus colegas são incríveis...",0,1,0,1,0,1,1
5,"chefe rude, me tratou mal quando cheguei na em...",1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...
6679,realmente muito longe a empresa,1,0,0,0,0,0,0
6700,"ações, comida no escritório, colegas interessa...",0,1,1,0,1,1,1
6701,cada vez mais complicado e lento no realizar p...,1,0,0,1,0,0,0
6728,"ambiente descontraídos, vários benefícios, bom...",0,1,1,1,0,1,1


In [ ]:
df_final_parte1 = df_final_parte1.reset_index(drop=True)
df_final_parte2 = df_final_parte2.reset_index(drop=True)

df_departamento = pd.concat([df_final_parte1, df_final_parte2], ignore_index=True)

In [ ]:
df_departamento = df_departamento[['Comentário','Work/Life Balance', 'Culture & Values', 'Diversity & Inclusion', 'Career Opportunities', 'Compensation and Benefits', 'Senior Management']]

In [ ]:
df_departamento

,Comentário,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"cultura, reconhecimento de melhorarias, todos ...",1,1,1,0,1,0
1,muitas mudanças em muito pouco tempo,0,0,0,1,0,1
2,era uma empresa ótima para se trabalhar.,0,0,0,0,0,0
3,"a empresa se perdeu, tocadores de bumbo que sã...",1,1,1,1,1,1
4,ambiente de trabalho amistoso e 100% em casa.,1,1,1,0,1,1
...,...,...,...,...,...,...,...
6362,realmente muito longe a empresa,0,0,0,0,0,0
6363,"ações, comida no escritório, colegas interessa...",1,1,0,1,1,1
6364,cada vez mais complicado e lento no realizar p...,0,0,1,0,0,0
6365,"ambiente descontraídos, vários benefícios, bom...",1,1,1,0,1,1


In [ ]:
target_list = colunas_para_transformar

In [ ]:
df_departamento_train, df_departamento_temp = train_test_split(df_departamento, test_size = 0.2, random_state=42)
df_departamento_eval, df_departamento_test = train_test_split(df_departamento_temp, test_size = 0.5, random_state=42)

In [ ]:
df_departamento_train.shape

(5093, 7)

In [ ]:
df_departamento_eval.shape

(637, 7)

In [ ]:
df_departamento_test.shape

(637, 7)

In [ ]:
df_departamento_train = df_departamento_train.reset_index(drop=True)
df_departamento_train

,Comentário,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"todos possíveis, impossível mensurar o trauma ...",1,1,1,1,1,1
1,"falta benefícios, regime presencial total ou q...",1,1,1,0,0,1
2,clima de competição. correria! obrigatorio o i...,1,0,0,0,1,0
3,muito trabalho para 1 pessoa sozinha fazer,0,0,0,0,0,0
4,bom primeiro elogiar a empresa e que e inovado...,1,1,1,1,1,1
...,...,...,...,...,...,...,...
5088,salário e benefício . não paga ticket aos aten...,1,1,1,1,1,1
5089,muita pressão e humilhação . muitas pessoas dã...,1,1,0,1,1,0
5090,"pagamentos em dia, boa infraestrutura física c...",0,0,1,0,0,0
5091,você consegue crescer dentro da empresa. o sal...,1,1,1,1,1,0


In [ ]:
df_departamento_eval = df_departamento_eval.reset_index(drop=True)
df_departamento_eval

,Comentário,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,não tem vale refeição infelizmente,0,0,0,0,0,0
1,"estresse coletivo, clima pesado, falta de moti...",1,0,0,0,1,1
2,empresas sensacional de se trabalhar recomendo...,1,1,1,1,1,1
3,"é uma empresa jovem, com lideranças presentes ...",0,0,0,0,1,0
4,"empresa longe, mas boa estrutura pra home",0,0,0,0,0,0
...,...,...,...,...,...,...,...
632,empresa tem uma remuneração compatível com mer...,1,0,0,0,1,0
633,muda constantemente de rota para atender as ne...,1,1,0,1,0,1
634,amigos leais. horário. líderes. crescimento.,0,1,1,1,0,0
635,home office na pandemia dificulta integração c...,0,0,0,0,0,0


In [ ]:
df_departamento_test = df_departamento_test.reset_index(drop=True)
df_departamento_test

,Comentário,Work/Life Balance,Culture & Values,Diversity & Inclusion,Career Opportunities,Compensation and Benefits,Senior Management
0,"a vantagem é no recrutamento, poucas fases, se...",0,0,1,0,0,0
1,"falta de empatia, respeito com o próximo, não ...",0,0,0,0,0,0
2,"muitas pessoas engajadas, competentes e parcei...",1,1,1,1,1,1
3,"organizada,etica,justa,possibilita o crescimen...",1,1,1,1,1,1
4,muitas ligações onde o cliente pode ser mal ed...,0,0,0,0,1,1
...,...,...,...,...,...,...,...
632,"autonomia e colaboração, espirito de equipe",0,1,0,0,1,0
633,a volta para o escritório foi feita de modo de...,1,1,1,1,1,1
634,salario fixo auxilio deslocamento celular linh...,0,0,1,0,0,0
635,algumas pessoas e alguns beneficios,0,0,0,0,0,0


In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 1e-05

In [ ]:
tokenizer_dep = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
class departamento_Dataset(Dataset):
  def __init__(self,textos,rotulos,tokenizer,max_len):
    self.textos = textos
    self.rotulos = rotulos
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.textos)

  def __getitem__(self, index):
    texto = str(self.textos[index])
    texto_lixo = " ".join(texto.split())

    encoded_texto = self.tokenizer.encode_plus(
        texto,
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    return{
        'inputs_ids': encoded_texto['input_ids'].flatten(),
        'attention_mask': encoded_texto['attention_mask'].flatten(),
        'labels': torch.FloatTensor(self.rotulos[index])
    }

In [ ]:
train_dep_dataset = departamento_Dataset(textos=df_departamento_train['Comentário'],
                                         rotulos=df_departamento_train[target_list].values,
                                         tokenizer = tokenizer_dep,
                                         max_len = MAX_LEN
                                         )

In [ ]:
eval_dep_dataset = departamento_Dataset(textos=df_departamento_eval['Comentário'],
                                         rotulos=df_departamento_eval[target_list].values,
                                         tokenizer = tokenizer_dep,
                                         max_len = MAX_LEN
                                         )

In [ ]:
test_dep_dataset = departamento_Dataset(textos=df_departamento_test['Comentário'],
                                         rotulos=df_departamento_test[target_list].values,
                                         tokenizer = tokenizer_dep,
                                         max_len = MAX_LEN
                                         )

In [ ]:
train_dep_dataloader = DataLoader(
    train_dep_dataset,
    shuffle=True,
    batch_size = TRAIN_BATCH_SIZE,
    num_workers=0
)

eval_dep_dataloader = DataLoader(
    eval_dep_dataset,
    shuffle=False,
    batch_size = EVAL_BATCH_SIZE,
    num_workers=0
)

test_dep_dataloader = DataLoader(
    test_dep_dataset,
    shuffle=False,
    batch_size = EVAL_BATCH_SIZE,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
class BERTimbau(nn.Module):
  def __init__(self):
    super(BERTimbau, self).__init__()
    self.bert_model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased', return_dict=True)
    self.dropout = nn.Dropout(0.3)
    self.linear = nn.Linear(768, 6)

  def forward(self, input_ids, attention_mask):
    output = self.bert_model(input_ids, attention_mask)
    output_dropout = self.dropout(output.pooler_output)
    output = self.linear(output_dropout)
    return output

model_dep = BERTimbau()
model_dep.to(device)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BERTimbau(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
def loss_fn(outputs, labels):
  return nn.BCEWithLogitsLoss()(outputs, labels)

optimizer = torch.optim.Adam(params=model_dep.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_model(n_epochs, train_dep_dataloader, eval_dep_dataloader, model_dep, optimizer):
  valid_loss_min = np.Inf

  for epoch in range(1,n_epochs+1):
    train_loss = 0.0
    eval_loss = 0.0

    model_dep.train()
    for index, batch in enumerate(train_dep_dataloader):
      input_ids = batch['inputs_ids'].to(device, dtype=torch.long)
      attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
      labels = batch['labels'].to(device, dtype=torch.float)
      outputs = model_dep(input_ids, attention_mask)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss = train_loss + (1/(index+1)*(loss.item()-train_loss))

    model_dep.eval()
    with torch.no_grad():
      for index, batch in enumerate(eval_dep_dataloader):
        input_ids = batch['inputs_ids'].to(device, dtype=torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.float)
        outputs = model_dep(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        eval_loss = eval_loss + (1/(index+1)*(loss.item()-eval_loss))

  print(f"Epoch: {epoch}, Train Loss: {train_loss}, Eval Loss: {eval_loss}")

  return model_dep

In [ ]:
trained_model = train_model(EPOCHS, train_dep_dataloader, eval_dep_dataloader, model_dep, optimizer)

Epoch: 30, Train Loss: 0.06145428866147994, Eval Loss: 1.2656763017177584


In [ ]:
row = df_departamento_test.iloc[115][['Comentário', 'Work/Life Balance', 'Culture & Values', 'Diversity & Inclusion',
                                     'Career Opportunities', 'Compensation and Benefits', 'Senior Management']]
row

Comentário                   pagamento em dia , e benefícios também.
Work/Life Balance                                                  0
Culture & Values                                                   1
Diversity & Inclusion                                              0
Career Opportunities                                               1
Compensation and Benefits                                          0
Senior Management                                                  0
Name: 115, dtype: object

In [ ]:
df_departamento_test['Comentário'][115]

'pagamento em dia , e benefícios também.'

In [ ]:
exemplo = 'ambiente saudável pouca valorização dos profissionais por desempenho.'

encoded_exemplo = tokenizer.encode_plus(
        exemplo,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

trained_model.eval()
with torch.no_grad():

    input_ids = encoded_exemplo['input_ids'].to(device, dtype=torch.long)
    attention_mask = encoded_exemplo['attention_mask'].to(device, dtype=torch.long)
    output = model_dep(input_ids, attention_mask)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)

[[0.9654850363731384, 0.9964728951454163, 0.15795588493347168, 0.9305922389030457, 0.9909974336624146, 0.9981874823570251]]
